### 목표
- 버섯의 특징(22가지 컬럼)을 활용해서 독성/식용 버섯을 분류해 나가보자
- 과대적합 제어 (4가지 방식) / 특성의 중요도 판별(중요 특성일수록 탑노드에 위치시킴) / 의사결정나무 모델 사용



In [7]:
#의사결정 라이브러리
import pandas as pd
import matplotlib.pyplot as plt

#머신러닝도구
from sklearn.model_selection import train_test_split # train test 분리도구
from sklearn.tree import DecisionTreeClassifier #라벨이 2개 식용과 독성 2개이기에 


In [8]:
#데이터 불러오기 csv
data=pd.read_csv('data/mushroom.csv')

In [9]:
# 전체 데이터 보는 기능 모든 컬럼을 다보여줍니다. 
pd.set_option('display.max_columns',None) 

data.head()

,poisonous,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


In [10]:
#데이터 크기확인
data.shape
#특성 (22개)+ labelData (1개)

(8124, 23)

In [11]:
#데이터의 정보확인
data.info()
#결측치 없고  ->굿~ 
#데이터 타입-> 문자열이기에 수치화로 인코딩해줘야합니다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   poisonous                 8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [12]:
#데이터 분리하기
X=data.iloc[:,1:]
y=data['poisonous']
#X=data.drop('poisonous',axis=1) 이렇게도 정답과 문제데이터 분리


In [13]:
print(X.shape)
print(y.shape)

(8124, 22)
(8124,)


In [14]:
#기술통계량 확인
X.describe()
#데이터타입이 문자열 형태이기때문에 문자열에대한 기술통계량을 제공
#count: 단순카운트개수
#unique: 등장한 알파벳의 개수 (중복되지않은 유일한값을 출력)
#top:제일많이 등장한 알파벳
#freq => top.freq 제일많이 등장한 알파벳의 등장 빈도수

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,6,4,10,2,9,2,2,2,12,2,5,4,4,9,9,1,4,3,5,9,6,7
top,x,y,n,f,n,f,c,b,b,t,b,s,s,w,w,p,w,o,p,w,v,d
freq,3656,3244,2284,4748,3528,7914,6812,5612,1728,4608,3776,5176,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [15]:
#label 값의 횟수 확인하기
y.value_counts()
#클래스 2개 
#e: 식용 p:독성

e    4208
p    3916
Name: poisonous, dtype: int64

### 데이터 전처리
- 문자데이터 -> 숫자데이터 (Encoding)
- 숫자데이터 -> 문자데이터 (Binning)
- 인코딩 방법 2가지 
    1. 원핫인코딩  X_one_hot=pd.get_dummies(데이터) : 값의 크고 작음의 의미가 없을때 사용 (우선순위X)
            1)기술통계량에서 unique한 개수만큼의 컬럼을 생성 
            2)일치하는 값에는 1을 아닌컬럼에는 0을 주는 방식으로 인코딩 진행
          
          - <분류 기준 분류하는 범주의 우선순위가있을경우 >
          - 상 버섯갓모양(아무거나해도됨)// 하 성적 a,b,c,d( 우선순위가있음)
          
    2. 레이블인코딩 :값의 크고 작음의 의미가 있을때 사용 (우선순위O)
            1) 
            2)
    
    

In [16]:
X_one_hot=pd.get_dummies(X)
print(X_one_hot)

      cap-shape_b  cap-shape_c  cap-shape_f  cap-shape_k  cap-shape_s  \
0               0            0            0            0            0   
1               0            0            0            0            0   
2               1            0            0            0            0   
3               0            0            0            0            0   
4               0            0            0            0            0   
...           ...          ...          ...          ...          ...   
8119            0            0            0            1            0   
8120            0            0            0            0            0   
8121            0            0            1            0            0   
8122            0            0            0            1            0   
8123            0            0            0            0            0   

      cap-shape_x  cap-surface_f  cap-surface_g  cap-surface_s  cap-surface_y  \
0               1              0          

In [17]:
X_train,X_test,y_train,y_test=train_test_split(X_one_hot,y,test_size=0.3,random_state=7)

In [18]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5686, 117)
(2438, 117)
(5686,)
(2438,)


### 모델링
- 모델객체 생성
- 학습 및 예측
- 평가

In [19]:
#모델 생성 
tree_model=DecisionTreeClassifier(max_depth=4, 
    min_samples_split=5,
    min_samples_leaf=5,
    )
# CTRL+SHIFT 무슨 하이퍼파라미터가있는지 찾아볼수있어요~ 

In [20]:
tree_model.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=4, min_samples_leaf=5, min_samples_split=5)

In [21]:
pre=tree_model.predict(X_test)

In [22]:
tree_model.score(X_test,y_test)

0.9930270713699754

### 특성선택(feature selection)
- 지도학습모델에서는 데이터간 각 특성들의 중요도를 출력해 볼 수 있다.
- 정답에 얼마나 영향을 미치는지 수치로 확인이 가능
- 0~1 사이의 중요도를 가진다 (1에 가까울수록 중요한 특성!)
- 모든 중요도의 합이 1이 됨
- tree모델은 내부적으로 특성의 중요도가 높은 조건을 선택하여 탑노드에 배치 

In [25]:
#전체 특성확인
X_one_hot.columns

Index(['cap-shape_b', 'cap-shape_c', 'cap-shape_f', 'cap-shape_k',
       'cap-shape_s', 'cap-shape_x', 'cap-surface_f', 'cap-surface_g',
       'cap-surface_s', 'cap-surface_y',
       ...
       'population_s', 'population_v', 'population_y', 'habitat_d',
       'habitat_g', 'habitat_l', 'habitat_m', 'habitat_p', 'habitat_u',
       'habitat_w'],
      dtype='object', length=117)

In [28]:
#줄어들어있는 데이터 확인방법
pd.set_option('display.max_seq_items',None)

In [30]:
#tree_model.feature_importances_
fi=tree_model.feature_importances_

In [52]:
#전체행데이터 출력방법 pd.set_option
pd.set_option('display.max_rows',None)
fi1=pd.DataFrame(fi,index=X_one_hot.columns,columns=['특성중요도'])

In [51]:
fi1.sort_values(by='특성중요도',ascending=False)
#리버스는 numpy에서 했습니다 여긴 ㄴㄴ

,특성중요도
odor_n,0.625805
stalk-root_c,0.186492
stalk-surface-below-ring_y,0.100906
spore-print-color_r,0.037182
odor_l,0.025770
bruises_t,0.017589
stalk-color-above-ring_n,0.006083
gill-size_n,0.000174
stalk-color-below-ring_y,0.000000
stalk-color-below-ring_o,0.000000


 ### 교차검증
 - 개념: 모델의 일반화성능 측정방법
       - 모델이 데이터에 대해서 얼마나 높은 정확도를 가지는지 평가할 때,
       여러번 나누어서 평가하는 것이 더 신뢰도가 높기 때문에
       - 방법: 한전된 데이터 안에서 훈련세트와 테스트세트를 한번 더 나누어 여러겹 나누어 평가한다.
       -우리가 만든 모델의 일반화 성능은 어느 정도일까? -> 교차검증

In [56]:
#교차검증 
# 일반화가 됐다는건? 어떤 새로운 데이터가 들어오더라도 비슷한 정확도를 내는것.
from sklearn.model_selection import cross_val_score
#cross_val_score(적용할 모델이름, 훈련 데이터, 답데이터, cv )
result=cross_val_score(tree_model,X_train,y_train,cv=5)
result
#tree_model: 어떤데이터가 들어ㅗ아도 비슷한 성능을 내고있음을 확인
# 모두 1에 가까운 정확도를 내고있고-> 일반화성능이 괜찮은편 

array([0.99297012, 0.98680739, 0.9885664 , 0.99120493, 0.99472296])